In [1]:
import torch
import datasets

In [2]:
label2id = {
	'xIntent': 0,
	'xReact': 1,
	'xAttr': 2,
	'xEffect': 3,
	'xWant': 4,
	'xNeed': 5
}
id2label = {
	0: 'xIntent',
	1: 'xReact',
	2: 'xAttr',
	3: 'xEffect',
	4: 'xWant',
	5: 'xNeed'
}

In [3]:
from datasets import load_dataset

ds = load_dataset("allenai/soda")

In [4]:
unused = list(ds['train'].features.keys())
unused.remove("relation")
unused.remove("narrative")
lim_ds = ds.remove_columns(unused)


In [5]:
cast_ds = lim_ds.cast(datasets.Features(
	{
		"relation": datasets.ClassLabel(names=['xIntent', 'xReact', 'xAttr', 'xEffect', 'xWant', 'xNeed'], id=[0,1,2,3,4,5]),
		"narrative": datasets.Value(dtype='string', id=None)
	}
))
cast_ds['train'].features['relation'].str2int('xWant')
ren_ds = cast_ds.rename_columns({'relation': 'labels', 'narrative': 'input_ids'})
ren_ds

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids'],
        num_rows: 1191582
    })
    validation: Dataset({
        features: ['labels', 'input_ids'],
        num_rows: 146346
    })
    test: Dataset({
        features: ['labels', 'input_ids'],
        num_rows: 148968
    })
})

In [ ]:
from transformers import LlamaForCausalLM, AutoTokenizer, LlamaForSequenceClassification

model = LlamaForCausalLM.from_pretrained(
    "BlackSamorez/TinyLlama-1_1B-Chat-v1_0-AQLM-2Bit-1x16-hf",
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes=True,
	torch_dtype="auto", device_map="auto", low_cpu_mem_usage=True
)
tokenizer = AutoTokenizer.from_pretrained("BlackSamorez/TinyLlama-1_1B-Chat-v1_0-AQLM-2Bit-1x16-hf")

In [7]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): QuantizedLinear()
          (k_proj): QuantizedLinear()
          (v_proj): QuantizedLinear()
          (o_proj): QuantizedLinear()
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): QuantizedLinear()
          (up_proj): QuantizedLinear()
          (down_proj): QuantizedLinear()
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): Linear(in_features=2048, out_features=32000, bias=False)
)

In [10]:
output = model.generate(tokenizer("No! I don't want to go!", return_tensors="pt")["input_ids"].cuda(), max_new_tokens=10)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [44]:
output = model.generate(tokenizer("No! I don't want to go!", return_tensors="pt")["input_ids"].cuda(), min_new_tokens=128, max_new_tokens=256)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [45]:
print(tokenizer.decode(output[0]))

<s> No! I don't want to go!
I don't want to go!
I don't want to go!
I don't want to go!
I don't want to go!
I don't want to go!
I don't want to go!
I don't want to go!
I don't want to go!
I don't want to go!
I don't want to go!
I don't want to go!
I don't want to go!
I don't want to go!
I don't want to go!
I don't want to go!
I don't want to go!
I don't want to go!
I don't want to go!
I don't want to go!
I don't want to go!
I don't want to go!
I don't want to go!
I don't want to go!
I don't want to go!
I don't want to go!
I don't want to go!
I don't want to go!
I don't want to go!
I don'


In [15]:
model.num_parameters()

333409280

In [6]:
from transformers import LlamaForCausalLM, AutoTokenizer, LlamaForSequenceClassification, LlamaConfig

model = LlamaForSequenceClassification.from_pretrained(
    "BlackSamorez/TinyLlama-1_1B-Chat-v1_0-AQLM-2Bit-1x16-hf",
	config=LlamaConfig.from_json_file("./llama_config.json"),
    ignore_mismatched_sizes=True,
	torch_dtype="auto", device_map="auto", low_cpu_mem_usage=True
)
tokenizer = AutoTokenizer.from_pretrained("BlackSamorez/TinyLlama-1_1B-Chat-v1_0-AQLM-2Bit-1x16-hf")

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at BlackSamorez/TinyLlama-1_1B-Chat-v1_0-AQLM-2Bit-1x16-hf and are newly initialized: ['score.codebooks', 'score.codes', 'score.scales']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
model.train()

LlamaForSequenceClassification(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048, padding_idx=2)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaFlashAttention2(
          (q_proj): QuantizedLinear()
          (k_proj): QuantizedLinear()
          (v_proj): QuantizedLinear()
          (o_proj): QuantizedLinear()
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): QuantizedLinear()
          (up_proj): QuantizedLinear()
          (down_proj): QuantizedLinear()
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (score): QuantizedLinear()
)

In [8]:
for layer in model.modules():
	print(layer.training)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [9]:
torch.nn.CrossEntropyLoss()(torch.tensor([[0,0,0,1,0,0],[0,0,0,1,0,0]], dtype=torch.float16, device="cuda"), torch.tensor([1,1], device="cuda:0"))

tensor(2.0430, device='cuda:0', dtype=torch.float16)

In [28]:
from aqlm import QuantizedLinear
QuantizedLinear(16, 8,8,1,16,16).state_dict()

OrderedDict([('codebooks',
              tensor([[[[0., 0., 0.,  ..., 0., 0., 0.]],
              
                       [[0., 0., 0.,  ..., 0., 0., 0.]],
              
                       [[0., 0., 0.,  ..., 0., 0., 0.]],
              
                       ...,
              
                       [[0., 0., 0.,  ..., 0., 0., 0.]],
              
                       [[0., 0., 0.,  ..., 0., 0., 0.]],
              
                       [[0., 0., 0.,  ..., 0., 0., 0.]]],
              
              
                      [[[0., 0., 0.,  ..., 0., 0., 0.]],
              
                       [[0., 0., 0.,  ..., 0., 0., 0.]],
              
                       [[0., 0., 0.,  ..., 0., 0., 0.]],
              
                       ...,
              
                       [[0., 0., 0.,  ..., 0., 0., 0.]],
              
                       [[0., 0., 0.,  ..., 0., 0., 0.]],
              
                       [[0., 0., 0.,  ..., 0., 0., 0.]]],
              
      

In [19]:
model.score.nbits_per_codebook

16

In [10]:
model(
	input_ids = torch.LongTensor([tokenizer.build_inputs_with_special_tokens(tokenizer.convert_tokens_to_ids(tokenizer.tokenize("I want pumpkin pie!!!!")))]).cuda(),
	labels = torch.LongTensor([4]).cuda()
)


tensor([[0., 0., 0., 0., 0., 0.]], device='cuda:0', dtype=torch.float16,
       grad_fn=<ViewBackward0>) tensor([4], device='cuda:0')


RuntimeError: size mismatch (got input: [6], target: [1])

In [29]:
from transformers import LlamaForCausalLM, AutoTokenizer, LlamaForSequenceClassification, LlamaConfig

model = LlamaForCausalLM.from_pretrained(
    "BlackSamorez/TinyLlama-1_1B-Chat-v1_0-AQLM-2Bit-1x16-hf",
	config=LlamaConfig.from_json_file("./llama_config.json"),
    ignore_mismatched_sizes=True,
	torch_dtype="auto", device_map="auto", low_cpu_mem_usage=True
)
tokenizer = AutoTokenizer.from_pretrained("BlackSamorez/TinyLlama-1_1B-Chat-v1_0-AQLM-2Bit-1x16-hf")

In [38]:
model(
	input_ids = torch.LongTensor([tokenizer.build_inputs_with_special_tokens(tokenizer.convert_tokens_to_ids(tokenizer.tokenize("I want pumpkin pie!!!!")))]).cuda(),
	labels = torch.LongTensor([1, 306, 864, 282, 3427, 9089, 5036, 6824, 6824]).cuda()
)


CausalLMOutputWithPast(loss=tensor(4.5837, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[[  0.6606,   1.3945,   0.2908,  ...,  -1.4600,  -0.6411,   0.3057],
         [ -9.0234,  -9.1094,   4.6680,  ...,  -4.6094,  -7.5664,  -4.9102],
         [ -8.9297,  -9.0703,   5.1016,  ...,  -4.1406,  -8.3750,  -4.5156],
         ...,
         [ -7.7227,  -7.8984,   6.3047,  ...,  -5.4258, -10.7500,  -2.6523],
         [ -9.0781,  -9.1172,  11.1562,  ...,  -5.1875, -10.5547,  -5.2578],
         [ -9.2734,  -9.4062,  11.6406,  ...,  -5.1797, -10.6719,  -3.9355]]],
       device='cuda:0', grad_fn=<ToCopyBackward0>), past_key_values=((tensor([[[[ 1.9211e-02, -3.7048e-02,  1.1246e-02,  ..., -1.2733e-02,
            4.2558e-04, -9.5596e-03],
          [ 4.1895e-01,  3.0469e-01,  1.9055e-01,  ...,  8.6609e-02,
            4.5105e-02,  7.9407e-02],
          [ 5.8008e-01,  1.6565e-01,  2.0581e-01,  ..., -8.3160e-03,
            3.1555e-02,  1.5640e-03],
          ...,
          [-1.6504e-

## Attempts to debug aqlm library

In [9]:
from aqlm.inference_kernels.cuda_kernel import CUDA_KERNEL
CUDA_KERNEL.code2x8_matmat

<module 'codebook_cuda' from 'C:\\Users\\picokatx\\AppData\\Local\\torch_extensions\\torch_extensions\\Cache\\py311_cu118\\codebook_cuda\\codebook_cuda.pyd'>

In [38]:
import os
from torch.utils.cpp_extension import load
CUDA_KERNEL = load(
    name="codebook_cuda",
	
	is_python_module=True,
	is_standalone=False,
    sources=[os.path.join("D:/projects/affect/TUCORE-GCN/AQLM/inference_lib/src/aqlm/inference_kernels", "cuda_kernel.cpp"), os.path.join("D:/projects/affect/TUCORE-GCN/AQLM/inference_lib/src/aqlm/inference_kernels", "cuda_kernel.cu")],
)

ImportError: DLL load failed while importing codebook_cuda_v2: The specified module could not be found.

In [7]:
import os
from torch.utils.cpp_extension import load
CUDA_KERNEL = load(
    name="codebook_cudaa",
    extra_cflags=['/std:c++17'],
	build_directory="D:/projects/affect/TUCORE-GCN/AQLM/inference_lib/src/aqlm/inference_kernels/build",
    sources=[os.path.join("D:/projects/affect/TUCORE-GCN/AQLM/inference_lib/src/aqlm/inference_kernels", "cuda_kernel.cpp"), os.path.join("D:/projects/affect/TUCORE-GCN/AQLM/inference_lib/src/aqlm/inference_kernels", "cuda_kernel.cu")],
)

RuntimeError: Error building extension 'codebook_cudaa': [1/3] C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.3\bin\nvcc --generate-dependencies-with-compile --dependency-output cuda_kernel.cuda.o.d -Xcudafe --diag_suppress=dll_interface_conflict_dllexport_assumed -Xcudafe --diag_suppress=dll_interface_conflict_none_assumed -Xcudafe --diag_suppress=field_without_dll_interface -Xcudafe --diag_suppress=base_class_has_different_dll_interface -Xcompiler /EHsc -Xcompiler /wd4190 -Xcompiler /wd4018 -Xcompiler /wd4275 -Xcompiler /wd4267 -Xcompiler /wd4244 -Xcompiler /wd4251 -Xcompiler /wd4819 -Xcompiler /MD -DTORCH_EXTENSION_NAME=codebook_cudaa -DTORCH_API_INCLUDE_EXTENSION_H -Id:\projects\affect\TUCORE-GCN\.venv\Lib\site-packages\torch\include -Id:\projects\affect\TUCORE-GCN\.venv\Lib\site-packages\torch\include\torch\csrc\api\include -Id:\projects\affect\TUCORE-GCN\.venv\Lib\site-packages\torch\include\TH -Id:\projects\affect\TUCORE-GCN\.venv\Lib\site-packages\torch\include\THC "-IC

In [2]:
import os
os.environ['CUDA_PATH']

'C:\\Program Files\\NVIDIA GPU Computing Toolkit\\CUDA\\v12.3'

In [12]:
type()

str

In [6]:
from torch import library
dir(library.impl)

['__annotations__',
 '__builtins__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__getstate__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [1]:
from aqlm.inference_kernels.cuda_kernel import CUDA_KERNEL
CUDA_KERNEL

<module 'codebook_cuda' from 'C:\\Users\\picokatx\\AppData\\Local\\torch_extensions\\torch_extensions\\Cache\\py311_cu118\\codebook_cuda\\codebook_cuda.pyd'>

In [1]:
import os
from torch.utils.cpp_extension import load
CUDA_KERNEL = load(
    name="codebook_cuda",
    extra_cflags=['/std:c++17'],
    sources=[os.path.join("D:/projects/affect/TUCORE-GCN/AQLM/inference_lib/src/aqlm/inference_kernels", "cuda_kernel.cpp"), os.path.join("D:/projects/affect/TUCORE-GCN/AQLM/inference_lib/src/aqlm/inference_kernels", "cuda_kernel.cu")],
)

ImportError: DLL load failed while importing codebook_cuda: The specified module could not be found.

In [8]:
CUDA_KERNEL.code1x16_matmat

<function codebook_cuda.PyCapsule.code1x16_matmat>

In [40]:
from torch.utils.cpp_extension import include_paths
include_paths(cuda=True)

['d:\\projects\\affect\\TUCORE-GCN\\.venv\\Lib\\site-packages\\torch\\include',
 'd:\\projects\\affect\\TUCORE-GCN\\.venv\\Lib\\site-packages\\torch\\include\\torch\\csrc\\api\\include',
 'd:\\projects\\affect\\TUCORE-GCN\\.venv\\Lib\\site-packages\\torch\\include\\TH',
 'd:\\projects\\affect\\TUCORE-GCN\\.venv\\Lib\\site-packages\\torch\\include\\THC',
 'C:\\Program Files\\NVIDIA GPU Computing Toolkit\\CUDA\\v12.3\\include']

In [39]:
void __cdecl code2x8_matvec_cuda(void const *,void const *,void *,void const *,int,int)" (?code2x8_matvec_cuda@@YAXPEBX0PEAX0HH@Z)
void __cdecl code2x8_matvec_cuda(void const * __restrict,void const * __restrict,void * __restrict,void const * __restrict,int,int)" (?code2x8_matvec_cuda@@YAXPEIBX0PEIAX0HH@Z)


SyntaxError: unterminated string literal (detected at line 1) (1358459814.py, line 1)

In [7]:
import importlib
importlib.util.spec_from_file_location("codebook_cuda", "C:/Users/picokatx/AppData/Local/torch_extensions/torch_extensions/Cache/py311_cu118/codebook_cuda/codebook_cuda.pyd")

ModuleSpec(name='codebook_cuda', loader=<_frozen_importlib_external.ExtensionFileLoader object at 0x000002340E542990>, origin='C:/Users/picokatx/AppData/Local/torch_extensions/torch_extensions/Cache/py311_cu118/codebook_cuda/codebook_cuda.pyd')

In [26]:
dir()

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_cached',
 '_set_fileattr',
 '_uninitialized_submodules',
 'cached',
 'has_location',
 'loader',
 'loader_state',
 'name',
 'origin',
 'parent',
 'submodule_search_locations']

In [29]:
importlib.util.spec_from_file_location("codebook_cuda", "C:/Users/picokatx/AppData/Local/torch_extensions/torch_extensions/Cache/py311_cu118/codebook_cuda/codebook_cuda.pyd").loader

In [24]:
import importlib

importlib.util.module_from_spec(importlib.util.spec_from_file_location("codebook_cuda", "C:/Users/picokatx/AppData/Local/torch_extensions/torch_extensions/Cache/py311_cu118/codebook_cuda/codebook_cuda.pyd"))


ImportError: DLL load failed while importing codebook_cuda: The specified module could not be found.

In [42]:
CUDA_KERNEL.replace("\\", '/')

'C:/Users/picokatx/AppData/Local/torch_extensions/torch_extensions/Cache/py311_cu118/codebook_cuda/codebook_cuda_v1.exe'

In [45]:
model.config

LlamaConfig {
  "_name_or_path": "BlackSamorez/TinyLlama-1_1B-Chat-v1_0-AQLM-2Bit-1x16-hf",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "id2label": {
    "0": "xIntent",
    "1": "xReact",
    "2": "xAttr",
    "3": "xEffect",
    "4": "xWant",
    "5": "xNeed"
  },
  "initializer_range": 0.02,
  "intermediate_size": 5632,
  "label2id": {
    "xAttr": 2,
    "xEffect": 3,
    "xIntent": 0,
    "xNeed": 5,
    "xReact": 1,
    "xWant": 4
  },
  "max_position_embeddings": 2048,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 22,
  "num_key_value_heads": 4,
  "pretraining_tp": 1,
  "quantization_config": {
    "in_group_size": 8,
    "linear_weights_not_to_quantize": [
      "model.layers.0.input_layernorm.weight",
      "model.layers.0.post_attention_layernorm.weight",
      "model.layers.1.input_layernorm.weig